In [1]:
import numpy as np
import pandas as pd
import crypto_object as co
import sys
from datetime import timedelta
import multiprocessing
from functools import partial
import pickle
import tqdm
import datetime


print(multiprocessing.cpu_count(), 'cores')

# We must import this explicitly, it is not imported by the top-level
# multiprocessing module.
import multiprocessing.pool
import time

from random import randint


class NoDaemonProcess(multiprocessing.Process):
    # make 'daemon' attribute always return False
    def _get_daemon(self):
        return False
    def _set_daemon(self, value):
        pass
    daemon = property(_get_daemon, _set_daemon)

# We sub-class multiprocessing.pool.Pool instead of multiprocessing.Pool
# because the latter is only a wrapper function, not a proper class.
class MyPool(multiprocessing.pool.Pool):

    Process = NoDaemonProcess

36 cores


/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


#  load coins

In [2]:
import os
import importlib
import crypto_object


def load_coins(dir_):
    """
    Function to take in a directory containing data
    and return a dictionary of `Coin` objects
    """
    
    # to store data
    coin_dict = {}
    
    for filename in os.listdir(dir_):
        
        # price data
        if filename.endswith("_price.csv"):
            
            # coin name
            coin_name = filename.split('_')[0]
            
            coin_dict[coin_name] = crypto_object.Coin(coin_name, dir_ + filename)
            
    return coin_dict
coins = load_coins('cryptocurrencypricehistory/')

## Helpers

In [12]:

def pathHelper(path_num, 
               starting_price, 
               N_days, 
               starting_index, 
               lookback,
               coin):
    # select a random return within lookback
    px = starting_price

    return_list = []
    for offset in np.arange(N_days):

        # pct returns to select from
        lb_start = starting_index + offset
        lb_end = lb_start + lookback
        possible_returns = coin.full_data.loc[lb_start:(lb_end + 1), 'Pct Returns'].values
        px = px * (1 + np.random.choice(possible_returns))
        return_list.append({'px':px,'N_days':N_days,'offset':offset,'path_num':path_num})
    return return_list

def random_paths(coin, current_date, expiry_date, lookback=90, N=100):
    """
    Function that produces random crypto price curves based on a lookback window
    of historical values

    
    Parameters
    ----------

        coin : crypto_object.Coin


        current_date : Datetime
            t = 0 for the operation
    
        expiry_date : Datetime
            expiry date of the option

        lookback : int
            number of previous days to draw returns from

        N : int
            number of streams to create
    """

    # create data frame to store values
    N_days = (expiry_date - current_date).days 
    paths = np.empty((N_days, N))

    # ensure that lookback window is possible
    longest_lookback = current_date - timedelta(days=lookback)
    if longest_lookback not in set(coin.full_data['Date']):
        print('ERROR: lookback window not possible with current start date')
        sys.exit(1)

    starting_index = np.argwhere(coin.full_data['Date'] == current_date)[0][0]
    starting_price = coin.full_data['Close'][starting_index]

    

    with MyPool(32) as p:              
        path_return = p.map(partial(pathHelper, 
               starting_price=starting_price, 
               N_days=N_days, 
               starting_index=starting_index, 
               lookback=lookback,
               coin=coin), np.arange(N))    
    
    
    for path_ in path_return:
        for each in path_:
            paths[int(each['N_days']) - int(each['offset']) - 1, int(each['path_num'])] = each['px'] 


    paths = pd.DataFrame(paths)
    paths.index = [current_date + timedelta(days=int(x)) for x in np.arange(N_days)]
    return(paths)





def empirical_method(coin, current_date, expiry_date, r, K, call_or_put, lookback=90, N=100):
    """
    Function that uses the empirical distribution
    of crypto prices to calculate option prices. 

    
    Parameters
    ----------

        coin : crypto_object.Coin


        current_date : Datetime
            t = 0 for the operation
    
        expiry_date : Datetime
            expiry date of the option

        r : float
            risk-free interest rate

        K : float
            exercise price

        call_or_put : str
            option type

        lookback : int
            number of previous days to draw returns from

        N : int
            number of streams to create
    """

    rps = random_paths(coin, current_date, expiry_date, lookback=lookback, N=N)

    # final value for paths
    final_values = rps.iloc[0, :]

    # payout given final values
    if call_or_put == 'call':
        payout = [np.clip(x - K, 0, None) for x in final_values]
    elif call_or_put == 'put':
        payout = [np.clip(K - x, 0, None) for x in final_values]


    N_days = (expiry_date - current_date).days
    discount_factor = (1 + ((N_days/365) * r))**-1

    # get empirical price
    emp_price = np.nanmean(payout) * discount_factor

    # ensure that these prices obey the call /put inequalities
    if call_or_put == 'call':

        # call bounds S_t > C_k > max(S_t - KZ(t, T), 0)
        if emp_price < np.nanmax([np.nanmean(rps.iloc[-1, :] - (K * discount_factor)), 0]):
            return np.nanmax([np.nanmean(rps.iloc[-1, :] - (K * discount_factor)), 0])
        elif emp_price > np.nanmean(rps.iloc[-1, :]):
            return np.nanmean(rps.iloc[-1, :])

    else:

        # max(KZ(t, T) - S_t, 0) < P_k < KZ(t,T)
        if emp_price < np.nanmax([np.nanmean((K * discount_factor) - rps.iloc[-1, :]), 0]):
            return np.nanmax([np.nanmean((K * discount_factor) - rps.iloc[-1, :]), 0])
        elif emp_price > (K * discount_factor):
            return (K * discount_factor)

    return emp_price

In [13]:
def get_options_strip(coin, current_date, r, N_iter):
    
    # near and far dates
    near_term = current_date + datetime.timedelta(days=5)
    next_term = current_date + datetime.timedelta(days=30)
    
    starting_index = np.argwhere(coin.full_data['Date'] == current_date)[0][0]
    starting_price = coin.full_data['Close'][starting_index]
    
    # produces ks to search over
    ks_near = np.linspace(starting_price * .9, starting_price * 1.1, 10)
    ks_next = np.linspace(starting_price * .8, starting_price * 1.2, 20)
    
    # near term options
    near_term_calls = np.empty((10, ))
    near_term_puts = np.empty((10, ))
    
    # next term options
    next_term_calls = np.empty((20, ))
    next_term_puts = np.empty((20, ))
    
    for i, k in enumerate(ks_near):
        near_term_calls[i] = empirical_method(coin, current_date, near_term, 0, k, 'call', lookback=5, N=N_iter)
        near_term_puts[i] = empirical_method(coin, current_date, near_term, 0, k, 'put', lookback=5, N=N_iter)
        
    for i, k in enumerate(ks_next):
        next_term_calls[i] = empirical_method(coin, current_date, near_term, 0, k, 'call', lookback=30, N=N_iter)
        next_term_puts[i] = empirical_method(coin, current_date, near_term, 0, k, 'put', lookback=30, N=N_iter)
        
    near_term_df = pd.DataFrame([near_term_calls, near_term_puts]).T
    near_term_df.columns = ['Calls', 'Puts']
    near_term_df.index = ks_near
    next_term_df = pd.DataFrame([next_term_calls, next_term_puts]).T
    next_term_df.columns = ['Calls', 'Puts']
    next_term_df.index = ks_next
    
    return (near_term_df, next_term_df)

In [5]:
from __future__ import division

def closest_call_or_put(val, array, call_or_put):
    
    try:
        # loop through array and return propery idx
        if call_or_put == 'call':
            return min(array[array - val > 0])
        else:
            return max(array[val - array > 0])
    except:
        return val
        

def cryptoVixHelper(i, coin, current_date, r, N_iter, N_paths):
    # near and next options strip
    near_strip, next_strip = get_options_strip(coin, current_date, r, N_paths)

    # get idx where calls and puts differ the least
    near_closest_idx = near_strip.mean(axis=1).idxmin()
    next_closest_idx = next_strip.mean(axis=1).idxmin()

    T_1 = 5/365
    T_2 = 30/365

    # near and next forward prices
    F_near = near_closest_idx + (np.exp(r * T_1) * \
                (near_strip.loc[near_closest_idx, 'Calls'] - \
                 near_strip.loc[near_closest_idx, 'Puts']))

    F_next = next_closest_idx + (np.exp(r * T_2) * \
                (next_strip.loc[next_closest_idx, 'Calls'] - \
                 next_strip.loc[next_closest_idx, 'Puts']))

    # near/next strikes to find K_0s
    ks_near = near_strip.index.values
    ks_next = next_strip.index.values



    k_0_near_call = closest_call_or_put(near_closest_idx, ks_near, 'call')
    k_0_near_put = closest_call_or_put(near_closest_idx, ks_near, 'put')

    k_0_next_call = closest_call_or_put(next_closest_idx, ks_next, 'call')
    k_0_next_put = closest_call_or_put(next_closest_idx, ks_next, 'put')

    # strikes given by np.linspace so the delta for strikes is constant
    # therefore sufficient to calc one delta
    delta_near = np.abs((ks_near[1] - ks_near[0]) / 2)
    delta_next = np.abs((ks_next[1] - ks_next[0]) / 2)

    # near and next strikes for calls and puts to be calculated
    ks_near_puts = ks_near[ks_near < k_0_near_put]
    ks_near_calls = ks_near[ks_near > k_0_near_call]
    ks_next_puts = ks_next[ks_next < k_0_next_put]
    ks_next_calls = ks_next[ks_next > k_0_next_call]

    # calculate near vol
    near_sum = 0
    for k in ks_near_puts:
        near_sum = near_sum + near_strip.loc[k, 'Puts'] / (k**2)
    for k in ks_near_calls:
        near_sum = near_sum + near_strip.loc[k, 'Calls'] / (k**2)

    sigma_2_near = (np.exp(r * T_1) * delta_near * (2 / T_1) * near_sum) - \
        ((1 / T_1) * (((F_near / np.nanmean([k_0_near_call, k_0_near_put])) - 1)**2))

    # calculate next vol
    next_sum = 0
    for k in ks_next_puts:
        next_sum += next_strip.loc[k, 'Puts'] / (k**2)
    for k in ks_next_calls:
        next_sum += next_strip.loc[k, 'Calls'] / (k**2)


    sigma_2_next = (np.exp(r * T_2) * delta_next * (2 / T_2) * next_sum) - \
        ((1 / T_2) * (((F_next / np.nanmean([k_0_next_call, k_0_next_put])) - 1)**2))

    return (100 * np.sqrt((T_1 * sigma_2_near) + (T_2 * sigma_2_next) * (365/30)))        



def cryptoVixParallel(coin, current_date, r, N_iter, N_paths):
    vix = np.empty((N_iter,))
    
    with MyPool(32) as p:
        
        
        vix = p.map(partial(cryptoVixHelper, coin=coin, 
                                              current_date=current_date, 
                                              r=r, 
                                              N_iter=N_iter, 
                                              N_paths=N_paths), np.arange(N_iter))
    
    return (np.nanmean(vix))

In [6]:
def coin_vix(coin, N_iter=4, N_paths=100):
    # create vix for a given coin
    
    # for replicability
    np.random.seed(109)
    dates = coin.full_data['Date'][:-31]
    vix = np.empty((len(dates,)))
    
    # create vix for each date
    for i in tqdm.trange(len(dates,)):
        
        vix[i] = cryptoVixParallel(coin, dates[i], 0, N_iter, N_paths)

    out_series = pd.Series(vix)
    out_series.index = dates
    return out_series

In [7]:
# pickler

def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)



In [8]:
coin_names = list(coins.keys())

coins1 = (coin_names[0:5])
coins2 = (coin_names[5:10])
coins3 = (coin_names[10:14])

In [9]:
#coin_vix(coins['qtum'])

In [11]:
def f(coin_name):
    try:
        N_paths = 4
        N_iter = 100
        x = coin_vix(coins[coin_name], N_iter=N_iter, N_paths=N_paths)
        filename = str(N_paths)+'_paths/'+str(N_iter)+'_iter_'+coin_name+'.pkl'
        save_object(x, filename)
        return (coin_name,' succeeded')
    except:
        return (coin_name,' failed' )
    
coin_vixes = []

if __name__ == '__main__':
    with MyPool(32) as p:
        coin_vixes = (p.map(f, coins2))

  3%|▎         | 47/1437 [00:54<26:58,  1.16s/it]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
  4%|▎         | 61/1729 [01:11<32:40,  1.18s/it]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:89: RuntimeWarning: Mean of empty slice
  4%|▎         | 64/1729 [01:15<32:37,  1.18s/it]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
  4%|▍         | 65/1631 [01:16<30:50,  1.18s/it]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipyke

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
  7%|▋         | 116/1631 [02:21<30:43,  1.22s/it]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 73%|███████▎  | 177/242 [02:23<00:52,  1.23it/s]]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
  7%|▋         | 120/1729 [02:24<32:16,  1.20s/it]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarn

 21%|██        | 344/1631 [05:59<22:24,  1.04s/it]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 24%|██▍       | 343/1437 [05:59<19:08,  1.05s/it]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 20%|█▉        | 338/1729 [06:02<24:51,  1.07s/it]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 21%|██        | 346/1631 [06:02<22:27,  1.05s/it]/home/ubuntu/anaconda3/lib/python3.6/s

 25%|██▌       | 366/1437 [06:21<18:37,  1.04s/it]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 21%|██        | 358/1729 [06:21<24:22,  1.07s/it]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 26%|██▌       | 368/1437 [06:23<18:34,  1.04s/it]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 21%|██        | 360/1729 [06:23<24:19,  1.07s/it]/home/ubuntu/anaconda3/lib/python3.6/s

 28%|██▊       | 401/1437 [06:52<17:46,  1.03s/it]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 48%|████▊     | 494/1026 [07:00<07:32,  1.18it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 29%|██▊       | 412/1437 [07:01<17:28,  1.02s/it]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 49%|████▊     | 498/1026 [07:02<07:28,  1.18it/s]/home/ubuntu/anaconda3/lib/python3.6/s

...

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 72%|███████▏  | 1182/1631 [18:02<06:51,  1.09it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 84%|████████▎ | 1201/1437 [18:03<03:32,  1.11it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 73%|███████▎  | 1183/1631 [18:03<06:50,  1.09it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeW

 87%|████████▋ | 1254/1437 [18:45<02:44,  1.11it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 87%|████████▋ | 1255/1437 [18:46<02:43,  1.11it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 88%|████████▊ | 1259/1437 [18:49<02:39,  1.11it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 76%|███████▌  | 1241/1631 [18:51<05:55,  1.10it/s]/home/ubuntu/anaconda3/lib/python3

 96%|█████████▌| 1376/1437 [20:25<00:54,  1.12it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 83%|████████▎ | 1354/1631 [20:27<04:11,  1.10it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 75%|███████▌  | 1303/1729 [20:28<06:41,  1.06it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 97%|█████████▋| 1390/1437 [20:36<00:41,  1.12it/s]/home/ubuntu/anaconda3/lib/python3

 81%|████████  | 1402/1729 [21:59<05:07,  1.06it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 82%|████████▏ | 1411/1729 [22:07<04:59,  1.06it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 91%|█████████ | 1484/1631 [22:18<02:12,  1.11it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 94%|█████████▍| 1535/1631 [23:02<01:26,  1.11it/s]/home/ubuntu/anaconda3/lib/python3

 96%|█████████▌| 1656/1729 [25:54<01:08,  1.07it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 96%|█████████▌| 1662/1729 [26:00<01:02,  1.07it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 96%|█████████▌| 1664/1729 [26:02<01:01,  1.07it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:73: RuntimeWarning: invalid value encountered in sqrt
 96%|█████████▋| 1666/1729 [26:04<00:59,  1.07it/s]/home/ubuntu/anaconda3/lib/python3